<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Option-A:-V1alpha1ScriptTemplate" data-toc-modified-id="Option-A:-V1alpha1ScriptTemplate-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Option A: <code>V1alpha1ScriptTemplate</code></a></span></li><li><span><a href="#Option-B:-closure" data-toc-modified-id="Option-B:-closure-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Option B: <code>closure</code></a></span></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from argo.workflows.sdk import Workflow

from argo.workflows.sdk.tasks import *
from argo.workflows.sdk.templates import *

In [3]:
import yaml

from pprint import pprint

from argo.workflows.sdk._utils import sanitize_for_serialization

---

In [4]:
!sh -c '[ -f "scripts.yaml" ] || curl -LO https://raw.githubusercontent.com/CermakM/argo-python-sdk/master/examples/scripts.yaml'

In [5]:
from pathlib import Path

manifest = Path("./scripts.yaml").read_text()
print(manifest)

# @file: scripts.yaml
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  name: scripts-python
  generateName: scripts-python-
spec:
  entrypoint: main
  templates:
  - name: main
    dag:
      tasks:
      - name: generate
        template: gen-random-int
      - name: print
        template: print-message
        arguments:
          parameters:
          - name: message
            value: "{{steps.generate.outputs.result}}"
        dependencies: [generate]

  - name: gen-random-int
    script:
      image: python:alpine3.6
      name: gen-random-int
      command: [python]
      source: |
        import random
        i = random.randint(1, 100)
        print(i)

  - name: print-message
    inputs:
      parameters:
      - name: message
    container:
      image: alpine:latest
      name: print-message
      command: [sh, -c]
      args: ["echo result was: {{inputs.parameters.message}}"]
status: {}



## Option A: `V1alpha1ScriptTemplate`

In [6]:
import textwrap

class ScriptsPython(Workflow):
    
    @task
    def generate(self) -> V1alpha1Template:
        return self.gen_random_int()
    
    @task
    @arguments.parameter(
        name="message",
        value="{{steps.generate.outputs.result}}"
    )
    @dependencies(["generate"])
    def print(self, message: str) -> V1alpha1Template:
        return self.print_message(message) 
    
    @template
    def gen_random_int(self) -> V1alpha1ScriptTemplate:
        source = textwrap.dedent("""\
          import random
          i = random.randint(1, 100)
          print(i)
        """)
      
        template = V1alpha1ScriptTemplate(
            image="python:alpine3.6",
            name="gen-random-int",
            command=["python"],
            source=source
        )
        
        return template
    
    @template
    @inputs.parameter(name="message")
    def print_message(self, message: str) -> V1Container:
        container = V1Container(
            image="alpine:latest",
            name="print-message",
            command=["sh", "-c"],
            args=["echo result was: {{inputs.parameters.message}}"],
        )
        
        return container
    
wf = ScriptsPython()
wf

{'api_version': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generate_name': 'scripts-python-', 'name': 'scripts-python'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'generate',
                                            'template': 'gen-random-int'},
                                           {'arguments': {'parameters': [{'name': 'message',
                                                                          'value': '{{steps.generate.outputs.result}}'}]},
                                            'dependencies': ['generate'],
                                            'name': 'print',
                                            'template': 'print-message'}]},
                         'name': 'main'},
                        {'name': 'gen-random-int',
                         'script': {'command': ['python'],
                                    'image': 'python:alpine3.6',
                                    'name': 'ge

In [7]:
print(wf.to_yaml())

api_version: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generate_name: scripts-python-
  name: scripts-python
spec:
  entrypoint: main
  templates:
  - dag:
      tasks:
      - name: generate
        template: gen-random-int
      - arguments:
          parameters:
          - name: message
            value: '{{steps.generate.outputs.result}}'
        dependencies:
        - generate
        name: print
        template: print-message
    name: main
  - name: gen-random-int
    script:
      command:
      - python
      image: python:alpine3.6
      name: gen-random-int
      source: 'import random

        i = random.randint(1, 100)

        print(i)

        '
  - container:
      args:
      - 'echo result was: {{inputs.parameters.message}}'
      command:
      - sh
      - -c
      image: alpine:latest
      name: print-message
    inputs:
      parameters:
      - name: message
    name: print-message



---

In [8]:
pprint(sanitize_for_serialization(wf))

{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generateName': 'scripts-python-', 'name': 'scripts-python'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'generate',
                                            'template': 'gen-random-int'},
                                           {'arguments': {'parameters': [{'name': 'message',
                                                                          'value': '{{steps.generate.outputs.result}}'}]},
                                            'dependencies': ['generate'],
                                            'name': 'print',
                                            'template': 'print-message'}]},
                         'name': 'main'},
                        {'name': 'gen-random-int',
                         'script': {'command': ['python'],
                                    'image': 'python:alpine3.6',
                                    'name': 'gen-

In [9]:
pprint(yaml.safe_load(manifest))

{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generateName': 'scripts-python-', 'name': 'scripts-python'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'generate',
                                            'template': 'gen-random-int'},
                                           {'arguments': {'parameters': [{'name': 'message',
                                                                          'value': '{{steps.generate.outputs.result}}'}]},
                                            'dependencies': ['generate'],
                                            'name': 'print',
                                            'template': 'print-message'}]},
                         'name': 'main'},
                        {'name': 'gen-random-int',
                         'script': {'command': ['python'],
                                    'image': 'python:alpine3.6',
                                    'name': 'gen-

In [10]:
from deepdiff import DeepDiff

diff = DeepDiff(sanitize_for_serialization(wf), yaml.safe_load(manifest))
diff

{}

In [11]:
assert not diff, "Manifests don't match."

## Option B: `closure`

In a more complicated example, passing around Python scripts gets quite tedious and unnecessary --- we work with Python anyway, so why don't we make use of it?

The way we can utilize a function *as is* is to use `clusure`s. A `closure` is a custom construct which hasn't got a counterpart in the Argo specification. It is, however, very useful when working with the Python SDK.

In [12]:
import textwrap

class ScriptsPython(Workflow):
    
    @task
    def generate(self) -> V1alpha1Template:
        return self.gen_random_int()
    
    @task
    @arguments.parameter(
        name="message",
        value="{{steps.generate.outputs.result}}"
    )
    @dependencies(["generate"])
    def print(self, message: str) -> V1alpha1Template:
        return self.print_message(message) 
    
    # A closure defines its own scope.
    # The code below gets trenslated into the same YAML specification as above,
    # that is, a `closure` decorator produces a `V1alpha1ScriptTemplate`.
    @closure(
        image="python:alpine3.6"
    )
    def gen_random_int():
        import random
        i = random.randint(1, 100)
        print(i)
    
    @template
    @inputs.parameter(name="message")
    def print_message(self, message: str) -> V1Container:
        container = V1Container(
            image="alpine:latest",
            name="print-message",
            command=["sh", "-c"],
            args=["echo result was: {{inputs.parameters.message}}"],
        )
        
        return container
    
wf = ScriptsPython()
wf

99


{'api_version': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generate_name': 'scripts-python-', 'name': 'scripts-python'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'generate',
                                            'template': 'gen-random-int'},
                                           {'arguments': {'parameters': [{'name': 'message',
                                                                          'value': '{{steps.generate.outputs.result}}'}]},
                                            'dependencies': ['generate'],
                                            'name': 'print',
                                            'template': 'print-message'}]},
                         'name': 'main'},
                        {'name': 'gen-random-int',
                         'script': {'command': ['python'],
                                    'image': 'python:alpine3.6',
                                    'name': 'ge

In [13]:
print(wf.to_yaml())

api_version: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generate_name: scripts-python-
  name: scripts-python
spec:
  entrypoint: main
  templates:
  - dag:
      tasks:
      - name: generate
        template: gen-random-int
      - arguments:
          parameters:
          - name: message
            value: '{{steps.generate.outputs.result}}'
        dependencies:
        - generate
        name: print
        template: print-message
    name: main
  - name: gen-random-int
    script:
      command:
      - python
      image: python:alpine3.6
      name: gen-random-int
      source: 'import random

        i = random.randint(1, 100)

        print(i)

        '
  - container:
      args:
      - 'echo result was: {{inputs.parameters.message}}'
      command:
      - sh
      - -c
      image: alpine:latest
      name: print-message
    inputs:
      parameters:
      - name: message
    name: print-message



---

In [14]:
pprint(sanitize_for_serialization(wf))

{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generateName': 'scripts-python-', 'name': 'scripts-python'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'generate',
                                            'template': 'gen-random-int'},
                                           {'arguments': {'parameters': [{'name': 'message',
                                                                          'value': '{{steps.generate.outputs.result}}'}]},
                                            'dependencies': ['generate'],
                                            'name': 'print',
                                            'template': 'print-message'}]},
                         'name': 'main'},
                        {'name': 'gen-random-int',
                         'script': {'command': ['python'],
                                    'image': 'python:alpine3.6',
                                    'name': 'gen-

In [15]:
pprint(yaml.safe_load(manifest))

{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generateName': 'scripts-python-', 'name': 'scripts-python'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'generate',
                                            'template': 'gen-random-int'},
                                           {'arguments': {'parameters': [{'name': 'message',
                                                                          'value': '{{steps.generate.outputs.result}}'}]},
                                            'dependencies': ['generate'],
                                            'name': 'print',
                                            'template': 'print-message'}]},
                         'name': 'main'},
                        {'name': 'gen-random-int',
                         'script': {'command': ['python'],
                                    'image': 'python:alpine3.6',
                                    'name': 'gen-

In [16]:
from deepdiff import DeepDiff

diff = DeepDiff(sanitize_for_serialization(wf), yaml.safe_load(manifest))
diff

{}

In [17]:
assert not diff, "Manifests don't match."